In [1]:
import os
gpu_device = 2    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7f3bd79f4f90; to 'numba.cuda.cudadrv.driver.Device' at 0x7f3bd79add30>

In [2]:
import pandas as pd
import re
import random
import numpy as np
import math
import pickle
from sklearn.metrics import classification_report, accuracy_score
from datasets import Dataset,concatenate_datasets,load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%run ../utils/data.ipynb
%run ../utils/mt5-small_data-answer.ipynb

In [4]:

# Cargar modelo y tokenizador
model_name = "luqh/ClinicalT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs,targets = flatten_and_filter_dataset(ground_truth,report_data)  


#ahora estaba a 1e-5 y 7 epoochs
training_args = TrainingArguments(
    
    evaluation_strategy="epoch",
    # eval_steps=10,
    
    learning_rate=1e-4,  # Lower for fine-tuning without losing generalization
    fp16=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    
    num_train_epochs=7,  # Shorter fine-tuning stage
    weight_decay=0.05,  # Lower weight decay to preserve learned features
    # save_total_limit=2,
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
dataset_final=pd.DataFrame.from_dict(inputs,orient='index')
targets=pd.DataFrame.from_dict(targets,orient='index')
dataset_final.columns=["text"]
targets.columns=["label"]


In [6]:
accuracies,output_dic=cross_validation(dataset_final,targets,training=True,low_beams=False,testing=False)

fold 0
3636 405


/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/modeling_flax_pytorch_utils.py:455: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  pt_model_dict[flax_key] = torch.from_numpy(flax_tensor)
All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mcarrilero/pytorch_env/li

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 3636
})
tensor([[11860,    10,   405,  ...,     0,     0,     0],
        [11860,    10,   405,  ...,     0,     0,     0],
        [11860,    10,   405,  ...,     0,     0,     0],
        ...,
        [11860,    10,   405,  ...,     0,     0,     0],
        [11860,    10,   125,  ...,     0,     0,     0],
        [11860,    10,   125,  ...,     0,     0,     0]])
tensor([[ 4273,     5,     1,  ...,  -100,  -100,  -100],
        [  442,    18, 26889,  ...,  -100,  -100,  -100],
        [  150,     5,     1,  ...,  -100,  -100,  -100],
        ...,
        [  150,     5,     1,  ...,  -100,  -100,  -100],
        [  954,   635,  5984,  ...,  -100,  -100,  -100],
        [   71,  4545,   205,  ...,  -100,  -100,  -100]])


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.110373
2,No log,0.061766
3,0.386200,0.052124
4,0.386200,0.050533
5,0.058300,0.053073
6,0.029100,0.050059


...

All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 396/396 [00:00<00:00, 2020.04 examples/s]


Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 3645
})
tensor([[11860,    10,    86,  ...,     0,     0,     0],
        [11860,    10,   125,  ...,     0,     0,     0],
        [11860,    10,   405,  ...,     0,     0,     0],
        ...,
        [11860,    10,   405,  ...,     0,     0,     0],
        [11860,    10,   405,  ...,     0,     0,     0],
        [11860,    10,   125,  ...,     0,     0,     0]])
tensor([[  646,  6748,     6,  ...,  -100,  -100,  -100],
        [  850,   226,   755,  ...,  -100,  -100,  -100],
        [  150,     5,     1,  ...,  -100,  -100,  -100],
        ...,
        [  150,     5,     1,  ...,  -100,  -100,  -100],
        [12265,     1,  -100,  ...,  -100,  -100,  -100],
        [  335,   226,   755,  ...,  -100,  -100,  -100]])


Epoch,Training Loss,Validation Loss
1,No log,0.111302
2,No log,0.065424
3,0.386700,0.072549
4,0.386700,0.054567
5,0.054700,0.057605
6,0.054700,0.057268
7,0.029400,0.060192


/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


age outputs
['50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '86.', 'no.', 'no.', 'no.', 'no.', 'no.', '59.', 'no.']
age labels
['50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '86.', 'no.', 'no.', 'no.', 'no.', 'no.', '59.', 'no.']
tipo outputs
['biopsy report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control o

In [9]:
accuracies

[0.9716981132075472,
 0.9528301886792453,
 0.9811320754716981,
 0.9516129032258065,
 0.9086021505376344,
 0.9301075268817204,
 0.9838709677419355,
 0.7580645161290323,
 0.978494623655914,
 0.9731182795698925,
 0.7228260869565217,
 0.8870967741935484,
 0.8548387096774194,
 0.8763440860215054,
 0.9032258064516129,
 0.9623655913978495,
 0.946236559139785,
 0.8602150537634409,
 0.18292682926829268,
 0.9146341463414634,
 0.8536585365853658,
 0.926829268292683,
 0.5609756097560976,
 0.8780487804878049,
 0.8658536585365854,
 0.7906976744186046]

In [15]:
first fold
4 epoch 1e-5
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.9500
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_margin: 0.8750
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.8750
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 0.8750
Accuracy for nodules_echo_stable: 1.0000


2 epoch 5e-6
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_shape: 0.8750
Accuracy for nodules_echo_margin: 0.7500
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 1.0000
Accuracy for nodules_echo_stable: 1.0000

SyntaxError: invalid syntax (1685721385.py, line 1)

In [ ]:
second fold
Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.8824
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.8889
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7778
Accuracy for nodules_echo_margin: 0.8000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.7000
Accuracy for nodules_echo_size: 0.8889
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9412
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.6111
Accuracy for nodules_echo_shape: 0.8000
Accuracy for nodules_echo_margin: 0.6000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.8000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9524
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.5882
Accuracy for symtomatic: 0.8235
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8889
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7222
Accuracy for nodules_echo_shape: 0.9000
Accuracy for nodules_echo_margin: 0.7000
Accuracy for nodules_echo_echogenicity: 0.6000
Accuracy for nodules_echo_location: 0.6000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

In [ ]:
third fold
Accuracy for age: 0.9500
Accuracy for tipo: 0.9500
Accuracy for tecnica: 0.9000
Accuracy for family: 1.0000
Accuracy for history: 0.7222
Accuracy for symtomatic: 0.9444
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8333
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.8333
Accuracy for nodules_echo_margin: 0.8889
Accuracy for nodules_echo_echogenicity: 0.8889
Accuracy for nodules_echo_location: 0.7778
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.9000
Accuracy for nodules_echo_stable: 0.8571

In [8]:
for i,tipo in enumerate(questions):
    print(tipo, accuracies[i])

age 0.9952830188679245
tipo 0.8773584905660378
tecnica 0.9858490566037735
family 0.8602150537634409
history 0.010752688172043012
symtomatic 0.010752688172043012
prosthesis 0.9946236559139785
birads 0.978494623655914
density_mammo 0.978494623655914
calcifications_benign 0.9838709677419355
density_echo 0.1956521739130435
ganglio_mamo 0.9731182795698925
lymph_benign 0.946236559139785
lymph_suspicious 0.956989247311828
parenchymal_distortion 0.016129032258064516
simple_cyst 0.9838709677419355
ductal_ectasia 0.9838709677419355
nodules_echo_num 0.9408602150537635
nodules_echo_description 0.012195121951219513
nodules_echo_shape 0.2804878048780488
nodules_echo_margin 0.08536585365853659
nodules_echo_echogenicity 0.12195121951219512
nodules_echo_location 0.04878048780487805
nodules_echo_size 0.2804878048780488
nodules_echo_known 0.8170731707317073
nodules_echo_stable 0.9302325581395349


In [ ]:
age 0.9952830188679245
tipo 0.9716981132075472
tecnica 0.9858490566037735
family 0.989247311827957
history 0.9301075268817204
symtomatic 0.0
prosthesis 0.9946236559139785
birads 0.967741935483871
density_mammo 0.9516129032258065
density_echo 0.6935483870967742
lymph_suspicious 0.9731182795698925
nodules_echo_num 0.7849462365591398
nodules_echo_size 0.7349397590361446
nodules_echo_known 0.7831325301204819
nodules_echo_stable nan

In [13]:
questions

['age',
 'tipo',
 'tecnica',
 'family',
 'history',
 'symtomatic',
 'prosthesis',
 'birads',
 'density_mammo',
 'calcifications_benign',
 'density_echo',
 'ganglio_mamo',
 'lymph_benign',
 'lymph_suspicious',
 'parenchymal_distortion',
 'simple_cyst',
 'ductal_ectasia',
 'nodules_echo_num',
 'nodules_echo_description',
 'nodules_echo_shape',
 'nodules_echo_margin',
 'nodules_echo_echogenicity',
 'nodules_echo_location',
 'nodules_echo_size',
 'nodules_echo_known',
 'nodules_echo_stable']

In [10]:
model_name="clinicalt5"+"1e47"+"no_copy"+"description"+"16batch"+"final"+"tokenized"
for tipo in questions:
    print(tipo)
    with open(f"results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
        pickle.dump(output_dic[tipo], file)
   

age
tipo
tecnica
family
history
symtomatic
prosthesis
birads
density_mammo
calcifications_benign
density_echo
ganglio_mamo
lymph_benign
lymph_suspicious
parenchymal_distortion
simple_cyst
ductal_ectasia
nodules_echo_num
nodules_echo_description
nodules_echo_shape
nodules_echo_margin
nodules_echo_echogenicity
nodules_echo_location
nodules_echo_size
nodules_echo_known
nodules_echo_stable


In [9]:
devices = cuda.gpus
dev=devices.current
cuda.select_device(dev.id)
cuda.close()

In [8]:
accuracies 4 epochs 1 e-5
[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

In [12]:
acc,acc_std
solo un fallo

(0.9952830188679245, 0.013636363636363624)

In [ ]:
2 epoch eta bikoitza
(0.9811320754716981, 0.026424662271916612)

In [ ]:
con 8 epochs

In [ ]:
con 4 epochs pero añadiendo los datos sin etiqueta.

In [10]:
5 epochs 1e-5
" Important! We do not want biopsy, BAG, post BAG, nodal staging ultrasound or BI-RADS 6 (Cancer patients) reports. Answer 'no' if it is any of these. answer: "
(0.9811320754716981, 0.022268088570756142)

(0.9811320754716981, 0.022268088570756142)